In [2]:
import pandas as pd
import json
import ollama
from datetime import datetime
from pydantic import BaseModel

# ----- Transaction Schema -----
class TransactionBase(BaseModel):
    id_bank_account: int
    date: datetime
    amount: float
    currency: str
    description: str
    recipient: str
    raw_data: str

In [3]:

def detect_delimiter(file_path):
    """Detects delimiter by analyzing the first few lines of the file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        sample = f.read(1024)  # Read a small portion of the file
    delimiters = [',', ';', '\t', '|']
    return max(delimiters, key=lambda d: sample.count(d))

In [31]:
file_revolut = 'revolut_chf.csv'
file_zkb = 'zkb.csv'

delimiter = detect_delimiter(file_revolut)
delimiter_zkb = detect_delimiter(file_zkb)

df = pd.read_csv(file_revolut, delimiter=delimiter, dtype=str)
df_zkb = pd.read_csv(file_zkb, delimiter=delimiter_zkb, dtype=str)

In [52]:
def generate_mapping_prompt(headers, sample_data):
    """Generates a highly strict prompt for AI-based column mapping, including a data sample."""
    return f'''
    You are an AI model tasked with mapping CSV column headers to a predefined schema.
    Your response **must be strictly** a JSON dictionary mapping the schema fields to the correct column names from the input data.
    
    The only acceptable format is:
    {{
        "date": "<original_column_name_1>",
        "amount": "<original_column_name_2>",
        "currency": "<original_column_name_3>",
        "description": "<original_column_name_4>",
        "recipient": "<original_column_name_5>"
    }}
    
    If no column from the input matches a given field, set its value to `null`.
    
    Instructions:
    - Your response **must** be a valid JSON dictionary and contain **only** the mappings.
    - **DO NOT** include any additional text, comments, explanations, or formatting issues.
    - The values in your response must be exact column names from the input headers or `null` if a match is not found.
    - Ensure that every key-value pair follows the expected schema strictly.
    
    Here is a small sample of the data for better understanding:
    {json.dumps(sample_data, indent=2, ensure_ascii=False)}
    
    Given these column headers: {headers}, provide only a JSON mapping in the exact format above.
    '''

In [66]:
def parse_csv(file_path, sample_size=5):
    """Loads CSV file with AI-assisted schema mapping, providing a sample of the data."""
    delimiter = detect_delimiter(file_path)
    df = pd.read_csv(file_path, delimiter=delimiter, dtype=str)
    
    # Select a small sample of data to provide context
    sample_data = df.sample(sample_size).to_dict(orient='records')
    
    prompt = generate_mapping_prompt(list(df.columns), sample_data)
    result = ollama.generate(model="llama3:8b", prompt=prompt)
    
    try:
        column_mapping = json.loads(result.get('response'))  # Ensure we only get pure JSON
        print(column_mapping)
    except json.JSONDecodeError:
        raise ValueError("AI response could not be parsed as JSON")
    
    # Create a new transactions DataFrame
    transactions_data = []
    for _, row in df.iterrows():
        transaction = {
            "date": row[column_mapping["date"]] if column_mapping["date"] else None,
            "amount": float(row[column_mapping["amount"]]) if column_mapping["amount"] else None,
            "currency": row[column_mapping["currency"]] if column_mapping["currency"] else None,
            "description": row[column_mapping["description"]] if column_mapping["description"] else None,
            "recipient": row[column_mapping["recipient"]] if column_mapping["recipient"] else None,
            "raw_data": json.dumps(row.to_dict(), ensure_ascii=False)
        }
        transactions_data.append(transaction)
    
    transactions_df = pd.DataFrame(transactions_data)
    transactions_df['date'] = pd.to_datetime(transactions_df['date'], errors='coerce')
    
    #return [TransactionBase(**row) for row in transactions_df.to_dict(orient='records')]
    return transactions_df

In [78]:
parse_csv("csvs/revolut_chf.csv")

{'date': 'Started Date', 'amount': 'Amount', 'currency': 'Currency', 'description': 'Description', 'recipient': None}


,date,amount,currency,description,recipient,raw_data
0,2025-02-03 14:30:27,-25.8,CHF,Coop,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Savings"",..."
1,2025-02-04 08:20:13,-7.8,CHF,SBB Ticket,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
2,2025-02-05 15:10:42,-12.5,CHF,Migros,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
3,2025-02-08 20:14:38,-67.3,CHF,Restaurant Alpenblick,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
4,2025-02-11 09:45:33,-15.4,CHF,Denner,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
5,2025-02-12 17:05:50,-3.2,CHF,EasyPark,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
6,2025-02-13 08:14:27,-5.5,CHF,Kiosk Zurich HB,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
7,2025-02-16 13:27:40,-120.0,CHF,Galaxus,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
8,2025-02-17 21:18:15,-199.5,CHF,Hotel Basel,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."
9,2025-02-18 19:32:20,-9.8,CHF,ZFV Mensa,None,"{""Type"": ""CARD_PAYMENT"", ""Product"": ""Current"",..."


In [79]:
parse_csv("csvs/zkb.csv")

{'date': 'Date', 'amount': 'Debit CHF', 'currency': None, 'description': 'Booking text', 'recipient': None}


/var/folders/w_/35n4rz_56djd5_n1l0741mjw0000gn/T/ipykernel_42992/3744394485.py:32: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  transactions_df['date'] = pd.to_datetime(transactions_df['date'], errors='coerce')


,date,amount,currency,description,recipient,raw_data
0,2025-02-19,3.20,None,Debit TWINT: SBB EASYRIDE ZURICH,None,"{""Date"": ""19.02.2025"", ""Booking text"": ""Debit ..."
1,2025-02-18,15.40,None,Debit TWINT: COOP CITY,None,"{""Date"": ""18.02.2025"", ""Booking text"": ""Debit ..."
2,2025-02-17,22.75,None,Debit TWINT: MIGROS ZURICH,None,"{""Date"": ""17.02.2025"", ""Booking text"": ""Debit ..."
3,2025-02-16,6.80,None,Debit TWINT: STARBUCKS ZURICH HB,None,"{""Date"": ""16.02.2025"", ""Booking text"": ""Debit ..."
4,2025-02-15,12.95,None,Debit TWINT: DENNER OERLIKON,None,"{""Date"": ""15.02.2025"", ""Booking text"": ""Debit ..."
5,2025-02-14,9.20,None,Debit TWINT: ZFV MENSA ETH ZURICH,None,"{""Date"": ""14.02.2025"", ""Booking text"": ""Debit ..."
6,2025-02-13,17.50,None,Debit TWINT: APOTHEKE ZURICH,None,"{""Date"": ""13.02.2025"", ""Booking text"": ""Debit ..."
7,2025-02-12,42.90,None,Debit TWINT: EASYJET ZRH-GVA,None,"{""Date"": ""12.02.2025"", ""Booking text"": ""Debit ..."
8,2025-02-11,4.50,None,Debit TWINT: TCHIBO BAHNHOFSTRASSE,None,"{""Date"": ""11.02.2025"", ""Booking text"": ""Debit ..."
9,2025-02-10,28.40,None,Debit TWINT: PIZZERIA DA MICHELE,None,"{""Date"": ""10.02.2025"", ""Booking text"": ""Debit ..."


In [80]:
parse_csv('csvs/n26.csv')

{'date': 'Booking Date', 'amount': 'Amount (CHF)', 'currency': 'Original Currency', 'description': 'Partner Name', 'recipient': None}


,date,amount,currency,description,recipient,raw_data
0,2025-02-02,-139.00,CHF,Coop City,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
1,2025-02-02,-5.25,CHF,Migros Zürich,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
2,2025-02-02,-75.00,CHF,UBS Bank,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
3,2025-02-02,-48.00,CHF,Kino Maxx,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
4,2025-02-02,-13.90,CHF,Hiltl Restaurant,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
5,2025-02-02,-16.80,CHF,Coffee Fellows,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
6,2025-02-02,-15.50,CHF,ApoZurich,None,"{""Booking Date"": ""2025-02-02"", ""Value Date"": ""..."
7,2025-02-03,-21.30,CHF,SBB Parking,None,"{""Booking Date"": ""2025-02-03"", ""Value Date"": ""..."
8,2025-02-03,-9.20,CHF,Jelmoli,None,"{""Booking Date"": ""2025-02-03"", ""Value Date"": ""..."
9,2025-02-04,-18.75,CHF,Bäckerei Buchmann,None,"{""Booking Date"": ""2025-02-04"", ""Value Date"": ""..."
